In [1]:
import pandas as pd
import numpy as np

from PIL import Image
import os
import scipy
import cv2

from tqdm import tqdm
from time import time
import json 

with open('config.json') as f_in:
    config = json.load(f_in)


def std_convoluted(im, N):
    """
    Calculate the standard deviation using a convolution operation.

    Parameters:
    - im (numpy.ndarray): Input image as a NumPy array.
    - N (int): Radius of the convolution kernel.

    Returns:
    - numpy.ndarray: Standard deviation image.
    """
    # Calculate squared image and initialize kernel
    im2 = im**2
    ones = np.ones(im.shape)
    kernel = np.ones((2*N+1, 2*N+1))

    # Convolve the image and squared image with the kernel
    s = scipy.signal.convolve2d(im, kernel, mode="same")
    s2 = scipy.signal.convolve2d(im2, kernel, mode="same")
    ns = scipy.signal.convolve2d(ones, kernel, mode="same")

    # Calculate the standard deviation image
    return np.sqrt((s2 - s**2 / ns) / ns)

def my_func(path_to_orthophoto_rgb):
    
    Image.MAX_IMAGE_PIXELS = None
    
    rgb_name = os.path.basename(path_to_orthophoto_rgb)
    dept = rgb_name[:2]
    year = rgb_name[3:7]
    path_to_orthophoto_irc = config["irc_path"][dept][year] + rgb_name[:config['irc_pos']] + '-IRC' + rgb_name[config['irc_pos']:]# config["irc_path"] + path_to_orthophoto_rgb[config['rgb_name_pos']:] # A Changer plus tard
    # mnhc_tiff = config['mnhc_path'] + "Diff_MNS_CORREL_1-0_LAMB93_20FD3525_" + str(path_to_orthophoto_rgb[config['rgb_coordinates_pos']:config['rgb_coordinates_pos']+8]).replace('-', '_') + ".tif"
    
    # position
    rgb_x = path_to_orthophoto_rgb[config['rgb_coordinates_pos']: config['rgb_coordinates_pos']+3]
    rgb_y = path_to_orthophoto_rgb[config['rgb_coordinates_pos']+4: config['rgb_coordinates_pos']+8]
    mnhc_dir_dept = config['mnhc_path']+dept+'_'+year+'/'
    
    # images IRC et RGB
    # ortho_irc_file = config["irc_path"] + '/' + dept + '-2020-0'+rgb_x+'-'+rgb_y+'-LA93-0M20-IRC-E080.jp2'
    # ouverture
    ortho_rgb = cv2.resize(np.asarray(Image.open(path_to_orthophoto_rgb)), (10000, 10000), interpolation=cv2.INTER_AREA)
    ortho_irc = cv2.resize(np.asarray(Image.open(path_to_orthophoto_irc)), (10000, 10000), interpolation=cv2.INTER_AREA) 
    ndvi = np.divide(ortho_irc[:,:,0]-ortho_irc[:,:,1],ortho_irc[:,:,0]+ortho_irc[:,:,1], where=(ortho_irc[:,:,0]+ortho_irc[:,:,1])!=0 )
    ortho_irc = None 
    
    if not os.path.exists(config['rgb_vignettes_path'] + f'rgb_{rgb_x}_{rgb_y}/'):
        os.makedirs(config['rgb_vignettes_path'] + f'rgb_{rgb_x}_{rgb_y}/')
    if not os.path.exists(config['mask_vignettes_path'] + f'mask_{rgb_x}_{rgb_y}/'):
        os.makedirs(config['mask_vignettes_path'] + f'mask_{rgb_x}_{rgb_y}/')

    # i = 0
    vignette_data = []
    for mnhc_x in tqdm(range(0,5)):
        for mnhc_y in range(0,5):
            mnhc_file = mnhc_dir_dept+'Diff_MNS_CORREL_1-0_LAMB93_20FD'+dept+'25_'+str(int(int(rgb_x)+mnhc_x))+'_'+str(int(int(rgb_y)-mnhc_y))+'.tif'
            
            if os.path.exists(mnhc_file):
                # print(mnhc_file)
                mnhc = np.asarray(Image.open(mnhc_file))
                w, h = mnhc.shape
                mask = np.logical_and(np.logical_and(mnhc > 3., 
                                                     np.logical_or(mnhc > 5., 
                                                                   std_convoluted(mnhc,N=10) > .5)
                                                    ),
                                      ndvi[mnhc_x*w:(mnhc_x+1)*w, mnhc_y*h:(mnhc_y+1)*h] > 0) \
                .astype('uint8')

                
                for x in range(0, w-config['img_size']+1, config['img_size']):
                    for y in range(0, h-config['img_size']+1, config['img_size']):
                        # print(x, y)
                        crop_rgb = ortho_rgb[mnhc_y*h+y: mnhc_y*h+y+config['img_size'], mnhc_x*w+x: mnhc_x*w+x+config['img_size'], :]
                        crop_mask = mask[y:y+config['img_size'], x:x+config['img_size']]

                        # filtre rgb
                        # enregistrement
                        pos = dept+'_'+str(int((int(rgb_x)+mnhc_x)*1000+x/2))+'_'+str(int((int(rgb_y)-mnhc_y)*1000+y/2))
                        
                        cv2.imwrite(config['rgb_vignettes_path'] + f'rgb_{rgb_x}_{rgb_y}/' + 'rgb_' + str(pos) + '.jpg', crop_rgb)
                        cv2.imwrite(config['mask_vignettes_path'] + f'mask_{rgb_x}_{rgb_y}/' + 'mask_'+ str(pos) + '.png', crop_mask*255)

                        # Calculate characteristics of the RGB crop and add to vignette_data
                        mask_2 = np.stack((crop_mask,)*3, axis=-1)
                        non_mask = np.logical_not(mask_2)
                        res = [pos, crop_mask.sum()] + \
                        np.mean(crop_rgb, axis=(0, 1), where=mask_2.astype(bool)).tolist() + \
                        np.mean(crop_rgb, axis=(0, 1), where=non_mask.astype(bool)).tolist() + \
                        np.std(crop_rgb, axis=(0, 1), where=mask_2.astype(bool)).tolist() + \
                        np.std(crop_rgb, axis=(0, 1), where=non_mask.astype(bool)).tolist()
                        vignette_data.append(res)
                
                # Si besoin de plus d'images, commenter le break
                break
    
    # Convert the vignette data to a Pandas DataFrame and return
    return pd.DataFrame(vignette_data, columns=('file', 'wood_size', 'w_mean_R', 'w_mean_G', 'w_mean_B', 'nw_mean_R', 'nw_mean_G', 'nw_mean_B', 'w_std_R', 'w_std_G', 'w_std_B', 'nw_std_R', 'nw_std_G', 'nw_std_B'))
                

In [2]:
path = 'donnees/BDORTHO/ORTHOHR_1-0_RVB-0M20_JP2-E080_LAMB93_D035_2020-01-01/ORTHOHR/1_DONNEES_LIVRAISON_2021-06-00121/OHR_RVB_0M20_JP2-E080_LAMB93_D35-2020/35-2020-0315-6780-LA93-0M20-E080.jp2'
df = my_func(path)

  0%|                                    | 0/5 [00:00<?, ?it/s]C:\Users\adelabrosse\.conda\envs\inrae_seg_env\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\adelabrosse\.conda\envs\inrae_seg_env\Lib\site-packages\numpy\core\_methods.py:184: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
C:\Users\adelabrosse\.conda\envs\inrae_seg_env\Lib\site-packages\numpy\core\_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\adelabrosse\.conda\envs\inrae_seg_env\Lib\site-packages\numpy\core\_methods.py:226: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
C:\Users\adelabrosse\.conda\envs\inrae_seg_env\Lib\site-packages\numpy\core\_methods.py:258: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
100%|███████

In [3]:
df

,file,wood_size,w_mean_R,w_mean_G,w_mean_B,nw_mean_R,nw_mean_G,nw_mean_B,w_std_R,w_std_G,w_std_B,nw_std_R,nw_std_G,nw_std_B
0,35_315000_6780000,10616,63.804258,74.364073,59.182084,120.482065,121.304862,102.990750,21.449493,18.847097,13.304599,55.201400,41.623148,43.996051
1,35_315000_6780128,4656,64.699098,76.916022,62.044029,87.417198,98.106603,72.804238,25.284402,23.189929,15.697924,30.739106,21.611210,19.710930
2,35_315000_6780256,234,84.243590,85.598291,72.658120,100.340403,110.879345,87.406542,25.243017,18.928141,13.913446,44.741457,31.875329,35.923007
3,35_315000_6780384,6172,75.243519,83.051037,68.246111,125.458308,124.557981,105.766357,26.677082,23.608470,18.092364,55.683681,42.272754,44.170127
4,35_315000_6780512,6771,71.049919,78.118594,63.397873,83.098222,91.564605,71.413069,25.478308,21.775739,15.786907,29.351996,22.434132,21.986896
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,35_319768_6780256,15724,68.554439,76.533897,67.248982,113.075705,114.847848,101.976913,32.801241,27.713955,19.808910,45.468546,39.289198,41.425113
241,35_319768_6780384,14480,64.182804,74.683494,62.870373,100.916797,108.831773,87.632090,31.462350,27.749536,19.599839,36.591143,30.670211,32.596305
242,35_319768_6780512,16304,69.903459,79.505336,65.413334,99.522871,105.894012,88.724590,30.706987,27.524117,21.489698,42.716193,35.516262,36.997131
243,35_319768_6780640,8759,66.097043,76.986985,64.983331,116.554820,117.654913,110.261585,28.320055,26.623528,20.230822,48.644890,42.856604,44.313154
